In [18]:
import pandas as pd
from datetime import datetime, timedelta
import holidays

In [19]:
# Carregar o DataFrame a partir do link raw do GitHub
df = pd.read_csv('estudo.csv', sep=';', encoding='latin1')

In [20]:
df

,NR_TRABALHO,Total_Horas,vinculo_atividade_planejamento,DEPE_atividade_planejamento,descricao_atividade_planejamento,tipo_atividade_planejamento,DT_INC_atividade_planejamento,DT_FIM_atividade_planejamento,horas_destinadas_trabalho
0,49320,2686,49823,8454,Comunicação de Resultados,Processo Corporativo,02DEC2024,04FEB2025,420
1,49320,2686,49812,8454,Pré-auditoria,Processo Corporativo,26FEB2024,07/jun/24,750
2,49320,2686,49822,8454,Execução,Processo Corporativo,10/jun/24,29/nov/24,1250
3,49320,2686,49824,8454,Supervisão,Processo Corporativo,26FEB2024,04FEB2025,266
4,49349,3010,49469,9647,Execução,Processo Corporativo,04/nov/24,27DEC2024,30
...,...,...,...,...,...,...,...,...,...
1551,54036,1929,54041,8454,Execução,Processo Corporativo,29SEP2025,06/nov/25,200
1552,54036,1929,54038,9889,Pré-auditoria,Processo Corporativo,25AUG2025,26SEP2025,525
1553,54036,1929,54040,9889,Execução,Processo Corporativo,29SEP2025,06/nov/25,609
1554,54036,1929,54042,9889,Comunicação de Resultados,Processo Corporativo,07/nov/25,21/nov/25,231


In [21]:
# Definir feriados do Brasil
br_holidays = holidays.Brazil()

In [22]:
# Certifique-se de que as colunas de data estão no formato datetime
def parse_dates(date_str):
    date_formats = ['%d%b%Y', '%d/%b/%y', '%d%b%y', '%d/%m/%y', '%d/%m/%Y', '%d%b%Y', '%d%b%y', '%d%b%Y']
    for fmt in date_formats:
        try:
            return pd.to_datetime(date_str, format=fmt, dayfirst=True)
        except (ValueError, TypeError):
            continue
    return pd.NaT


In [23]:
# Aplicar a função de análise de datas às colunas
df['DT_INC_atividade_planejamento'] = df['DT_INC_atividade_planejamento'].apply(parse_dates)
df['DT_FIM_atividade_planejamento'] = df['DT_FIM_atividade_planejamento'].apply(parse_dates)

# Verificar se as datas foram convertidas corretamente
if df['DT_INC_atividade_planejamento'].isnull().any() or df['DT_FIM_atividade_planejamento'].isnull().any():
    print("Atenção: Algumas datas não foram convertidas corretamente. Verifique os formatos de data.")


In [24]:
# Obter o ano mínimo e máximo das colunas de data
min_year = df['DT_INC_atividade_planejamento'].dt.year.min()
max_year = df['DT_FIM_atividade_planejamento'].dt.year.max()

# Criar a lista de anos dinamicamente
anos = list(range(min_year, max_year + 1))

In [25]:
anos

[2024, 2025, 2026]

In [26]:
# Para cada ano, calcule o número de dias úteis excluindo feriados
for ano in anos:
    inicio_ano = pd.Timestamp(f'{ano}-01-01')
    fim_ano = pd.Timestamp(f'{ano}-12-31')
    
    df[f'dias_uteis_{ano}'] = df.apply(
        lambda row: len(
            pd.bdate_range(
                start=max(row['DT_INC_atividade_planejamento'], inicio_ano), 
                end=min(row['DT_FIM_atividade_planejamento'], fim_ano),
                holidays=br_holidays
            )
        ) if pd.notnull(row['DT_INC_atividade_planejamento']) and pd.notnull(row['DT_FIM_atividade_planejamento']) and min(row['DT_FIM_atividade_planejamento'], fim_ano) >= max(row['DT_INC_atividade_planejamento'], inicio_ano) else 0,
        axis=1
    )

In [27]:
# Calcular o total de dias úteis no período total excluindo feriados
df['total_dias_uteis'] = df.apply(
    lambda row: len(
        pd.bdate_range(
            start=row['DT_INC_atividade_planejamento'], 
            end=row['DT_FIM_atividade_planejamento'],
            holidays=br_holidays
        )
    ) if pd.notnull(row['DT_INC_atividade_planejamento']) and pd.notnull(row['DT_FIM_atividade_planejamento']) else pd.NA,
    axis=1
)

In [28]:
# Evitar divisão por zero
df['total_dias_uteis'] = df['total_dias_uteis'].replace({0: pd.NA})

# Calcular horas por dia útil
df['horas_por_dia_util'] = df['horas_destinadas_trabalho'] / df['total_dias_uteis']

In [29]:
# Para cada ano, calcular as horas trabalhadas
for ano in anos:
    df[f'horas_{ano}'] = df[f'dias_uteis_{ano}'] * df['horas_por_dia_util']
    # Substituir valores nulos por zero
    df[f'horas_{ano}'] = df[f'horas_{ano}'].fillna(0)

# Selecionar as colunas para exibição, incluindo dias úteis por ano
colunas_resultado = ['NR_TRABALHO', 'DT_INC_atividade_planejamento', 'DT_FIM_atividade_planejamento', 'horas_destinadas_trabalho', 'total_dias_uteis'] + [f'dias_uteis_{ano}' for ano in anos] + [f'horas_{ano}' for ano in anos]

In [30]:
df.head(1)

,NR_TRABALHO,Total_Horas,vinculo_atividade_planejamento,DEPE_atividade_planejamento,descricao_atividade_planejamento,tipo_atividade_planejamento,DT_INC_atividade_planejamento,DT_FIM_atividade_planejamento,horas_destinadas_trabalho,dias_uteis_2024,dias_uteis_2025,dias_uteis_2026,total_dias_uteis,horas_por_dia_util,horas_2024,horas_2025,horas_2026
0,49320,2686,49823,8454,Comunicação de Resultados,Processo Corporativo,2024-12-02,2025-02-04,420,22,25,0,47,8.93617,196.595745,223.404255,0.0


In [32]:
df.query('DT_INC_atividade_planejamento >= "2024-12-30" and DT_FIM_atividade_planejamento <= "2025-01-17" and DEPE_atividade_planejamento == 9889')

,NR_TRABALHO,Total_Horas,vinculo_atividade_planejamento,DEPE_atividade_planejamento,descricao_atividade_planejamento,tipo_atividade_planejamento,DT_INC_atividade_planejamento,DT_FIM_atividade_planejamento,horas_destinadas_trabalho,dias_uteis_2024,dias_uteis_2025,dias_uteis_2026,total_dias_uteis,horas_por_dia_util,horas_2024,horas_2025,horas_2026
7,49349,3010,49353,9889,Comunicação de Resultados,Processo Corporativo,2024-12-30,2025-01-17,590,2,13,0,15,39.333333,78.666667,511.333333,0.0
326,51914,1178,51918,9889,Pré-auditoria,Processo Corporativo,2025-01-02,2025-01-17,50,0,12,0,12,4.166667,0.000000,50.000000,0.0


In [14]:
df.columns

Index(['NR_TRABALHO', 'Total_Horas', 'vinculo_atividade_planejamento',
       'DEPE_atividade_planejamento', 'descricao_atividade_planejamento',
       'tipo_atividade_planejamento', 'DT_INC_atividade_planejamento',
       'DT_FIM_atividade_planejamento', 'horas_destinadas_trabalho',
       'dias_uteis_2024', 'dias_uteis_2025', 'dias_uteis_2026',
       'total_dias_uteis', 'horas_por_dia_util', 'horas_2024', 'horas_2025',
       'horas_2026'],
      dtype='object')

- **Objetivo:** Determinar quantas horas de trabalho de um projeto são atribuídas a cada ano, considerando apenas os dias úteis (excluindo fins de semana e feriados).

- **Processo:**

  1. **Identificação do Período de Trabalho:**
     - Para cada projeto, identificamos as datas de início e fim para saber em quais anos o trabalho ocorre.

  2. **Cálculo dos Dias Úteis por Ano:**
     - Dentro desse período, calculamos quantos dias úteis (dias de trabalho) existem em cada ano.

  3. **Distribuição das Horas:**
     - Pegamos o total de horas destinadas ao projeto e distribuímos proporcionalmente entre os anos, de acordo com o número de dias úteis em cada ano.

---

**Explicação Simplificada:**

Imagine que você tem um projeto que começa em junho de 2024 e termina em fevereiro de 2025, com um total de **1.000 horas** de trabalho.

1. **Contagem dos Dias de Trabalho:**
   - **Em 2024:** Contamos quantos dias úteis há de junho a dezembro.
   - **Em 2025:** Contamos quantos dias úteis há de janeiro a fevereiro.

2. **Cálculo da Proporção de Horas:**
   - Suponha que haja **150 dias úteis** no total (100 em 2024 e 50 em 2025).
   - **Horas por dia útil:** Dividimos as 1.000 horas pelos 150 dias úteis, resultando em aproximadamente **6,67 horas por dia**.

3. **Distribuição das Horas por Ano:**
   - **Para 2024:** Multiplicamos as 6,67 horas/dia pelos 100 dias úteis, totalizando **667 horas**.
   - **Para 2025:** Multiplicamos as 6,67 horas/dia pelos 50 dias úteis, totalizando **333 horas**.



In [40]:
df_temp = df[['NR_TRABALHO','DEPE_atividade_planejamento', 'DT_INC_atividade_planejamento', 'DT_FIM_atividade_planejamento', 'horas_por_dia_util']].copy()

In [57]:
# Certifique-se de que as datas no DataFrame estão no formato datetime
df_temp['DT_INC_atividade_planejamento'] = pd.to_datetime(df_temp['DT_INC_atividade_planejamento'])
df_temp['DT_FIM_atividade_planejamento'] = pd.to_datetime(df_temp['DT_FIM_atividade_planejamento'])

# Obter o ano mínimo e máximo para definir o intervalo de feriados
data_inicio = df_temp['DT_INC_atividade_planejamento'].min()
data_fim = df_temp['DT_FIM_atividade_planejamento'].max()
anos = range(data_inicio.year, data_fim.year + 1)

# Criar a lista de feriados do Brasil no intervalo de anos relevante
br_holidays = holidays.Brazil(years=anos)
lista_feriados = list(br_holidays.keys())

In [58]:
lista_feriados

[datetime.date(2024, 1, 1),
 datetime.date(2024, 4, 21),
 datetime.date(2024, 5, 1),
 datetime.date(2024, 9, 7),
 datetime.date(2024, 10, 12),
 datetime.date(2024, 11, 2),
 datetime.date(2024, 11, 15),
 datetime.date(2024, 12, 25),
 datetime.date(2024, 3, 29),
 datetime.date(2024, 3, 31),
 datetime.date(2024, 5, 30),
 datetime.date(2024, 2, 14),
 datetime.date(2024, 2, 13),
 datetime.date(2025, 1, 1),
 datetime.date(2025, 4, 21),
 datetime.date(2025, 5, 1),
 datetime.date(2025, 9, 7),
 datetime.date(2025, 10, 12),
 datetime.date(2025, 11, 2),
 datetime.date(2025, 11, 15),
 datetime.date(2025, 12, 25),
 datetime.date(2025, 4, 18),
 datetime.date(2025, 4, 20),
 datetime.date(2025, 6, 19),
 datetime.date(2025, 3, 5),
 datetime.date(2025, 3, 4),
 datetime.date(2026, 1, 1),
 datetime.date(2026, 4, 21),
 datetime.date(2026, 5, 1),
 datetime.date(2026, 9, 7),
 datetime.date(2026, 10, 12),
 datetime.date(2026, 11, 2),
 datetime.date(2026, 11, 15),
 datetime.date(2026, 12, 25),
 datetime.date(2

In [60]:
# Função para gerar lista de dias úteis excluindo finais de semana e feriados
def gerar_dias_uteis(row):
    if pd.notnull(row['DT_INC_atividade_planejamento']) and pd.notnull(row['DT_FIM_atividade_planejamento']):
        return pd.date_range(
            start=row['DT_INC_atividade_planejamento'],
            end=row['DT_FIM_atividade_planejamento'],
            freq=cbd
        )
    else:
        return []

# Aplicar a função para gerar a lista de dias úteis
df_temp['dias_uteis'] = df_temp.apply(gerar_dias_uteis, axis=1)

NameError: name 'cbd' is not defined

In [42]:
df.query('NR_TRABALHO == 49349 and DEPE_atividade_planejamento == 9889')

,NR_TRABALHO,Total_Horas,vinculo_atividade_planejamento,DEPE_atividade_planejamento,descricao_atividade_planejamento,tipo_atividade_planejamento,DT_INC_atividade_planejamento,DT_FIM_atividade_planejamento,horas_destinadas_trabalho,dias_uteis_2024,dias_uteis_2025,dias_uteis_2026,total_dias_uteis,horas_por_dia_util,horas_2024,horas_2025,horas_2026
6,49349,3010,49352,9889,Execução,Processo Corporativo,2024-11-04,2024-12-27,1179,40,0,0,40,29.475000,1179.000000,0.000000,0.0
7,49349,3010,49353,9889,Comunicação de Resultados,Processo Corporativo,2024-12-30,2025-01-17,590,2,13,0,15,39.333333,78.666667,511.333333,0.0
8,49349,3010,49351,9889,Pré-auditoria,Processo Corporativo,2024-09-23,2024-11-01,931,30,0,0,30,31.033333,931.000000,0.000000,0.0
9,49349,3010,49354,9889,Supervisão,Processo Corporativo,2024-09-23,2025-01-17,270,72,13,0,85,3.176471,228.705882,41.294118,0.0


In [53]:
# Explodir o DataFrame para ter uma linha por dia útil por DEPE_atividade_planejamento e NR_TRABALHO
df_explodido = df_temp.explode('dias_uteis')

# Remover linhas com datas nulas (caso existam)
df_explodido = df_explodido.dropna(subset=['dias_uteis'])

# Selecionar as colunas relevantes, incluindo 'DEPE_atividade_planejamento' e 'NR_TRABALHO'
df_explodido = df_explodido[['DEPE_atividade_planejamento', 'NR_TRABALHO', 'dias_uteis', 'horas_por_dia_util']]

# Renomear a coluna 'dias_uteis' para 'data'
df_explodido.rename(columns={'dias_uteis': 'data'}, inplace=True)

# Converter a coluna 'data' para datetime, se necessário
df_explodido['data'] = pd.to_datetime(df_explodido['data'])

# Agrupar por 'data', 'DEPE_atividade_planejamento' e 'NR_TRABALHO' para obter o total de horas por dia
forecast_trb_horas_destinadas_trabalho = df_explodido.groupby(['data', 'DEPE_atividade_planejamento', 'NR_TRABALHO']).agg({'horas_por_dia_util': 'sum'}).reset_index()

# Renomear a coluna de horas
forecast_trb_horas_destinadas_trabalho.rename(columns={'horas_por_dia_util': 'total_horas_dia'}, inplace=True)

# Ordenar o DataFrame
forecast_trb_horas_destinadas_trabalho.sort_values(['data', 'DEPE_atividade_planejamento', 'NR_TRABALHO'], inplace=True)

In [54]:
# Configurar o pandas para exibir todas as linhas
pd.set_option('display.max_rows', None)

# Configurar o pandas para exibir todas as colunas
pd.set_option('display.max_columns', None)

# Opcional: Configurar o pandas para exibir o conteúdo completo de cada coluna, sem truncar
pd.set_option('display.max_colwidth', None)

In [56]:
forecast_trb_horas_destinadas_trabalho.query('total_horas_dia > 22')

,data,DEPE_atividade_planejamento,NR_TRABALHO,total_horas_dia
387,2024-09-02,9875,49487,23.416667
398,2024-09-03,9875,49487,23.416667
409,2024-09-04,9875,49487,23.416667
420,2024-09-05,9875,49487,23.416667
431,2024-09-06,9875,49487,23.416667
442,2024-09-09,9875,49487,23.416667
453,2024-09-10,9875,49487,23.416667
464,2024-09-11,9875,49487,23.416667
475,2024-09-12,9875,49487,23.416667
486,2024-09-13,9875,49487,23.416667


In [52]:
df[['NR_TRABALHO','DEPE_atividade_planejamento', 'DT_INC_atividade_planejamento', 'DT_FIM_atividade_planejamento', 'horas_por_dia_util']].query('DEPE_atividade_planejamento == 9889 and DT_INC_atividade_planejamento <= "2025-10-13" and DT_FIM_atividade_planejamento >= "2025-10-17" ')

,NR_TRABALHO,DEPE_atividade_planejamento,DT_INC_atividade_planejamento,DT_FIM_atividade_planejamento,horas_por_dia_util
290,51871,9889,2025-10-09,2025-11-25,11.882353
292,51871,9889,2025-09-03,2025-12-17,1.500000
424,52074,9889,2025-08-18,2025-11-14,3.230769
426,52074,9889,2025-08-18,2025-12-12,0.588235
561,52213,9889,2025-09-26,2025-10-24,11.904762
562,52213,9889,2025-07-07,2025-10-24,1.250000
859,52678,9889,2025-09-01,2025-11-14,3.636364
861,52678,9889,2025-08-04,2025-12-12,0.421053
872,52703,9889,2025-09-22,2025-10-17,19.700000
873,52703,9889,2025-06-09,2025-10-17,2.336842
